## Welcome to your notebook.


#### Run this cell to connect to your GIS and get started:

In [1]:
from arcgis.gis import GIS
gis = GIS("home")

/opt/conda/lib/python3.11/site-packages/arcgis/gis/__init__.py:731: UserWarning: You are logged on as reits041_UMN with an administrator role, proceed with caution.
  warnings.warn(


#### Now you are ready to start!

In [2]:
#Packages
import requests
import pandas as pd
import zipfile
import os
import io
from arcgis import features, GeoAccessor, GeoSeriesAccessor
from arcgis.features.analysis import join_features
from arcgis import features
from arcgis.features import FeatureLayerCollection
from arcgis.geometry import SpatialReference
from arcgis.gis import GIS
from arcgis.geometry import Geometry

In [3]:
request=requests.get('https://ndawn.ndsu.nodak.edu/table.csv?station=78&variable=ydmxt&ttype=yearly')
response=request.content
request=pd.read_csv(io.StringIO(response.decode('utf-8')), skiprows=[0,1,2,4])
sdf = GeoAccessor.from_xy(request, 'Longitude', 'Latitude')

In [4]:
arcrequest = requests.get('https://webgis.dot.state.mn.us/65agsf1/rest/services/sdw_govnt/COUNTY/FeatureServer/0/query?where=1%3D1&outSR=4326&f=geojson')
arcresponse = arcrequest.json()
arcdf=pd.json_normalize(arcresponse['features'])

arcdf['SHAPE'] = arcdf['geometry.coordinates'].apply(lambda coords: Geometry({
    "rings": coords,
    "spatialReference": {"wkid": 4326}
}))

In [5]:
print(arcdf.spatial.sr)
print(sdf.spatial.sr)

{'wkid': 4326}
{'wkid': 4326}


In [6]:
joined_sedf = sdf.spatial.join(arcdf, how='left', op='intersects')

In [7]:
joined_sedf.head()

,Station Name,Latitude,Longitude,Elevation,Year,Avg Max Temp,Number Missing,Number Estimated,SHAPE,index_right,type,geometry.type,geometry.coordinates,properties.COUNTY_NAME
0,Ada,47.32119,-96.51406,910,2008,48.805,0,0,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96....",66,Feature,Polygon,"[[[-96.66543704622316, 47.498946001559055], [-...",Norman
1,Ada,47.32119,-96.51406,910,2009,48.829,0,0,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96....",66,Feature,Polygon,"[[[-96.66543704622316, 47.498946001559055], [-...",Norman
2,Ada,47.32119,-96.51406,910,2010,52.036,0,0,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96....",66,Feature,Polygon,"[[[-96.66543704622316, 47.498946001559055], [-...",Norman
3,Ada,47.32119,-96.51406,910,2011,51.884,0,0,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96....",66,Feature,Polygon,"[[[-96.66543704622316, 47.498946001559055], [-...",Norman
4,Ada,47.32119,-96.51406,910,2012,55.731,0,0,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96....",66,Feature,Polygon,"[[[-96.66543704622316, 47.498946001559055], [-...",Norman


In [8]:
sj_fc=joined_sedf.spatial.to_featureclass('/arcgis/home/spatialjoin_sdf')

In [9]:

zip_file= '/arcgis/home/joined_sedf.zip'
file_path = os.path.join('arcgis/home',zip_file )
item_properties = {
    'title': 'Lab 1: NDAWN and Arc REST API Spatial Join',
    'tags': 'shapefile, weather, roadway',
    'type': 'Shapefile'
}

uploaded_item = gis.content.add(item_properties=item_properties, data=file_path)
uploaded_item.share(everyone=True)



Exception: Item 'joined_sedf.zip' already exists.
(Error Code: 409)